# Use DCP as initializer
---
## Define dataset

In [1]:
import numpy as np
from dcp import train, test
from data import SceneNet
from torch.utils.data import DataLoader
import sys
sys.path.append("dcp-master")
from model import DCP
import torch
import warnings
warnings.filterwarnings('ignore')


trainDataset = SceneNet(1024, "train")
valDataset = SceneNet(1024, "val")
train_loader = DataLoader(trainDataset, batch_size=32, shuffle=False, drop_last=False)
test_loader = DataLoader(valDataset, batch_size=1, shuffle=False, drop_last=False)
print(len(trainDataset))
print(len(valDataset))

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
299
299


## Define parameters

In [3]:
args = {
        "model_path": 'checkpoints/dcp_v1/models/model.best.t7',
        # "model_path": 'dcp-master/pretrained/dcp_v2.t7',
        "exp_name":"dcp_v1",
         "model":"dcp", 
         "emb_nn":"dgcnn", 
         "pointer":"identity", 
         "head":"svd", 
         "eval": True,
         'emb_dims': 512,
         'cycle': False,
         'use_sgd': False,
         'lr': 0.001,
         'epochs': 250,
         'n_blocks': 1,
         'dropout': 0.0,
         'ff_dims': 1024,
         'n_heads': 4,
         'use_sgd': False,
         'momentum': 0.9,
        }
net = DCP(args)

## DCP Training

In [3]:

train(args, net, train_loader, test_loader)

Use Adam


100%|██████████| 299/299 [00:02<00:00, 144.04it/s]


Epoch 0: -train- Loss: 140.252760,  -val- Loss: 20.018235


100%|██████████| 299/299 [00:01<00:00, 160.00it/s]


Epoch 1: -train- Loss: 82.563642,  -val- Loss: 39.954441


...

100%|██████████| 299/299 [00:02<00:00, 144.71it/s]


Epoch 10: -train- Loss: 59.681221,  -val- Loss: 37.592841


100%|██████████| 299/299 [00:02<00:00, 142.36it/s]


Epoch 11: -train- Loss: 57.215170,  -val- Loss: 27.649919


KeyboardInterrupt: 

## DCP Testing

In [4]:
net.load_state_dict(torch.load(args['model_path']), strict=False)

r, t = test(args, net, test_loader)
print(r[0])
print(t[0])

100%|██████████| 299/299 [00:06<00:00, 44.01it/s] 


==FINAL TEST==
A--------->B
EPOCH:: 1, Loss: 18.654200, Cycle Loss: 0.000000, MSE: 2.038408, RMSE: 1.427728, MAE: 1.028823, rot_MSE: 24.127089, rot_RMSE: 4.911933, rot_MAE: 3.244349, trans_MSE: 0.040257, trans_RMSE: 0.200640, trans_MAE: 0.131760
B--------->A
EPOCH:: 1, Loss: 18.654200, MSE: 2.038408, RMSE: 1.427728, MAE: 1.028205, rot_MSE: 24.172363, rot_RMSE: 4.916540, rot_MAE: 3.263173, trans_MSE: 0.039524, trans_RMSE: 0.198806, trans_MAE: 0.130251
[[ 0.99502903  0.08289655  0.05518261]
 [-0.08219942  0.99650586 -0.01479   ]
 [-0.05621544  0.01018047  0.9983666 ]]
[-0.40880036 -0.01387513  0.21633339]


## Visualization

In [7]:
import matplotlib.pyplot as plt
import k3d
from util import transform_point_cloud

def visualize_pointcloud(point_cloud1, point_size, point_cloud2=None, flip_axes=False, name='point_cloud', R=None, t=None):
    plot = k3d.plot(name=name, grid_visible=False, grid=(-0.55, -0.55, -0.55, 0.55, 0.55, 0.55))
    plt_points1 = k3d.points(positions=point_cloud1, point_size=point_size, color=0xd0d0d0)
    plot += plt_points1
    plt_points1.shader = '3d'
    if point_cloud2 is not None:
        plt_points2 = k3d.points(positions=point_cloud2, point_size=point_size, color=0x0dd00d)
        plot += plt_points2
        plt_points2.shader = '3d'
    plot.display()

def transform(point_cloud, R=None, t=None):
    t_broadcast = np.broadcast_to(t[:, np.newaxis], (3, point_cloud.shape[0]))
    return (R @ point_cloud.T + t_broadcast).T


src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[20]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src), torch.tensor(r[20]).unsqueeze(0), torch.tensor(t[20]).unsqueeze(0)).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(src.T, .03, target.T)

[[ 0.99980277 -0.00443393  0.0193574 ]
 [ 0.00360092  0.9990747   0.04285804]
 [-0.01952951 -0.04277989  0.9988936 ]]
[-0.0209675   0.00747702  0.02584793]


Output()

Output()

Output()

# ICP Calibration

In [5]:
from icp import test

valDataset = SceneNet(1024, "val", icp=True, r=r, t=t)
print(len(valDataset))
r_icp, t_icp = test(valDataset)

299


100%|██████████| 299/299 [18:26<00:00,  3.70s/it]

==FINAL TEST==
A--------->B
EPOCH:: -1, Loss: 0.064569, Cycle Loss: 0.000000, MSE: 0.012127, RMSE: 0.110121, MAE: 0.022179, rot_MSE: 26.837176, rot_RMSE: 5.180461, rot_MAE: 3.329714, trans_MSE: 0.060489, trans_RMSE: 0.245946, trans_MAE: 0.141536
B--------->A
EPOCH:: -1, Loss: 0.064569, MSE: 0.012127, RMSE: 0.110121, MAE: 0.021606, rot_MSE: 26.886614, rot_RMSE: 5.185230, rot_MAE: 3.342849, trans_MSE: 0.059994, trans_RMSE: 0.244936, trans_MAE: 0.140190


## Final Visualisation

In [8]:
src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[20]
print(rotation_ab)
print(translation_ab)
# print(points1.shape)
transformed_src = transform_point_cloud(torch.tensor(src).double(), torch.tensor(r_icp[20]).unsqueeze(0).double(), torch.tensor(t_icp[20]).unsqueeze(0).double()).T
transformed_src2 = transform_point_cloud(torch.tensor(src), torch.tensor(rotation_ab).unsqueeze(0), torch.tensor(translation_ab).unsqueeze(0)).T
# transformed_src1 = transform_point_cloud(torch.tensor(src), r1, t1).T
# visualize_pointcloud(target.T, .03, transformed_src1)
visualize_pointcloud(target.T, .03, transformed_src)
visualize_pointcloud(target.T, .03, transformed_src2)
visualize_pointcloud(src.T, .03, target.T)

[[ 0.99980277 -0.00443393  0.0193574 ]
 [ 0.00360092  0.9990747   0.04285804]
 [-0.01952951 -0.04277989  0.9988936 ]]
[-0.0209675   0.00747702  0.02584793]


Output()

Output()

Output()

## Export

In [10]:
"""Export to disk"""


def export_mesh_to_obj(path, vertices, faces, vertices2):
    """
    exports mesh as OBJ
    :param path: output path for the OBJ file
    :param vertices: Nx3 vertices
    :param faces: Mx3 faces
    :return: None
    """

    # write vertices starting with "v "
    # write faces starting with "f "

    # ###############
    # DONE: Implement
    v = ""
    f = ""
    v2 = ""
    file = open(path, 'w+')

    if vertices is not None:
        for vertice in vertices:
            v = v + "v "
            for i in vertice:
                v = v + str(i) + " "
            v = v + "0.407843 0.941176 0.917647\n"
            # v = v + "\n"
        
    count = 1
    if faces is not None:
        for face in faces:
            f = f + "f "
            for i in face:
                f = f + str(i+1) + " "
            f = f + "\n"

    if vertices2 is not None:
        for vertice2 in vertices2:
            v2 = v2 + "v "
            for i in vertice2:
                v2 = v2 + str(i[0]) + " "
            v2 = v2 + "0.98823529 0.5215686 0.317647\n"  
            # v2 = v2 + "\n"    
    file.write(v)
    file.write(v2)
    file.write(f)
    file.close()
        
    # ###############


def export_pointcloud_to_obj(path, pointcloud, pointcloud2=None):
    """
    export pointcloud as OBJ
    :param path: output path for the OBJ file
    :param pointcloud: Nx3 points
    :return: None
    """

    # ###############
    # DONE: Implement
    export_mesh_to_obj(path, pointcloud, None, pointcloud2)
    # ###############

# print(rigid_body_transformation_params)
valDataset = SceneNet(10000, "val", icp=True, r=r, t=t)
for i in range(len(valDataset)):
    src, target, rotation_ab, translation_ab, rotation_ba, translation_ba, euler_ab, euler_ba = valDataset[i]
    transformed_src = transform_point_cloud(torch.tensor(src).double(), torch.tensor(r_icp[i]).unsqueeze(0).double(), torch.tensor(t_icp[i]).unsqueeze(0).double()).T
    export_pointcloud_to_obj('56_'+str(i)+'.obj', target.T, np.array(transformed_src))